In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

In [3]:
excel_file = "Study Case DA.xlsx"

comp = pd.read_excel(excel_file, sheet_name="competencies_yearly")
perf = pd.read_excel(excel_file, sheet_name="performance_yearly")
psych = pd.read_excel(excel_file, sheet_name="profiles_psych")
employees = pd.read_excel(excel_file, sheet_name="employees")
strengths = pd.read_excel(excel_file, sheet_name="strengths")
dim_pillars = pd.read_excel(excel_file, sheet_name="dim_competency_pillars")
dim_grades = pd.read_excel(excel_file, sheet_name="dim_grades")

## STEP 1: Create Baseline from High Performers

In [4]:
# Filter 2025 data
perf_2025 = perf[perf['year'] == 2025]
comp_2025 = comp[comp['year'] == 2025]

# High performers: rating = 5
high_perf_ids = set(perf_2025[perf_2025['rating'] == 5.0]['employee_id'])
print(f"  ✓ High performers (rating 5): {len(high_perf_ids)} employees")

# Baseline Competencies
comp_high = comp_2025[comp_2025['employee_id'].isin(high_perf_ids)]
baseline_comp = comp_high.groupby('pillar_code')['score'].agg(['mean', 'std']).round(2)

print(f"  ✓ Baseline competencies: {len(baseline_comp)} pillars")
print("\n    Baseline Competency Scores (High Performers):")
print(baseline_comp)

# Baseline Psychometric
psych_high = psych[psych['employee_id'].isin(high_perf_ids)]

baseline_psych = pd.DataFrame({
    'mean': [
        psych_high['pauli'].mean(),
        psych_high['gtq'].mean(),
        psych_high['iq'].mean()
    ],
    'std': [
        psych_high['pauli'].std(),
        psych_high['gtq'].std(),
        psych_high['iq'].std()
    ]
}, index=['PAULI', 'GTQ', 'IQ']).round(2)

print(f"\n    Baseline Psychometric Scores (High Performers):")
print(baseline_psych)

  ✓ High performers (rating 5): 168 employees
  ✓ Baseline competencies: 10 pillars

    Baseline Competency Scores (High Performers):
             mean   std
pillar_code            
CEX          4.54  0.84
CSI          5.24  7.66
FTC          4.55  0.76
GDR          4.55  0.73
IDS          4.54  0.83
LIE          4.58  0.68
QDD          4.69  0.63
SEA          5.23  7.54
STO          5.11  7.52
VCU          5.20  7.57

    Baseline Psychometric Scores (High Performers):
         mean    std
PAULI   63.36  23.05
GTQ     28.22   6.56
IQ     109.10  18.47


## STEP 2: Define TV Match Calculation Functions

In [5]:
def calculate_tv_match(candidate_score, baseline_mean, baseline_std):
    """
    Calculate TV match rate using Z-score normalization.
    Formula: Match Rate = MAX(0, 100 - (|score - baseline_mean| / baseline_std * 10))
    Args:
        candidate_score: Individual's score
        baseline_mean: High performer average score
        baseline_std: High performer standard deviation
    
    Returns:
        Match rate (0-100%)
    """
    if pd.isna(candidate_score) or pd.isna(baseline_mean) or pd.isna(baseline_std):
        return 0
    
    if baseline_std == 0:
        return 100 if candidate_score == baseline_mean else 0
    
    distance = abs(candidate_score - baseline_mean) / baseline_std * 10
    return max(0, 100 - distance)


def calculate_competency_match(emp_id):
    """
    Calculate TGV match for Competency Excellence (50% weight).
    - Aggregates 10 competency pillars
    - Each pillar = 1 TV
    - Average of all TV match rates = TGV match rate
    """
    emp_comp = comp_2025[comp_2025['employee_id'] == emp_id]
    
    if emp_comp.empty:
        return {
            'competency_score': 0,
            'tv_details': []
        }
    
    tv_details = []
    match_rates = []
    
    for _, row in emp_comp.iterrows():
        pillar = row['pillar_code']
        score = row['score']
        
        if pillar in baseline_comp.index:
            baseline_mean = baseline_comp.loc[pillar, 'mean']
            baseline_std = baseline_comp.loc[pillar, 'std']
            
            tv_match = calculate_tv_match(score, baseline_mean, baseline_std)
            match_rates.append(tv_match)
            
            tv_details.append({
                'tv_code': pillar,
                'tv_name': pillar,
                'user_score': score,
                'baseline_score': baseline_mean,
                'tv_match_rate': round(tv_match, 2)
            })
    
    comp_score = round(np.mean(match_rates), 2) if match_rates else 0
    
    return {
        'competency_score': comp_score,
        'tv_details': tv_details
    }


def calculate_psychometric_match(emp_id):
    """
    Calculate TGV match for Cognitive Ability (25% weight).
    
    - 3 TVs: PAULI (15%), GTQ (7%), IQ (3%)
    - Weighted average of TV match rates = TGV match rate
    
    Psychometric Weights (within 25% TGV):
      PAULI: 15% / 25% = 60%
      GTQ:   7% / 25% = 28%
      IQ:    3% / 25% = 12%
    """
    emp_psych = psych[psych['employee_id'] == emp_id]
    
    if emp_psych.empty:
        return {
            'psychometric_score': 0,
            'tv_details': []
        }
    
    tv_details = []
    total_match = 0
    total_weight = 0
    
    # PAULI: Mental Speed & Accuracy (60% of psychometric TGV, 15% of final)
    tv_weights = {
        'pauli': ('PAULI', 'PAULI - Mental Speed & Accuracy', 0.60),
        'gtq': ('GTQ', 'GTQ - General Aptitude Test', 0.28),
        'iq': ('IQ', 'IQ - Intelligence Quotient', 0.12)
    }
    
    for var_key, (tv_code, tv_name, weight) in tv_weights.items():
        val = emp_psych[var_key].iloc[0]
        
        if not pd.isna(val) and tv_code in baseline_psych.index:
            baseline_mean = baseline_psych.loc[tv_code, 'mean']
            baseline_std = baseline_psych.loc[tv_code, 'std']
            
            tv_match = calculate_tv_match(val, baseline_mean, baseline_std)
            total_match += tv_match * weight
            total_weight += weight
            
            tv_details.append({
                'tv_code': tv_code,
                'tv_name': tv_name,
                'user_score': val,
                'baseline_score': baseline_mean,
                'tv_match_rate': round(tv_match, 2),
                'weight_in_tgv': weight
            })
    
    psych_score = round(total_match / total_weight, 2) if total_weight > 0 else 0
    
    return {
        'psychometric_score': psych_score,
        'tv_details': tv_details
    }


def calculate_behavioral_match(emp_id):
    """
    Calculate TGV match for Behavioral Strengths (20% weight).
    
    - 2 TVs: Thinker_Cluster, Doer_Cluster
    - Binary: 100 if theme in top 5, 0 otherwise
    - Average of TV match rates = TGV match rate
    
    Thinker Themes: Intellection, Analytical, Strategic, Futuristic
    Doer Themes: Activator, Responsibility, Self-Assurance, Belief
    """
    emp_strengths = strengths[strengths['employee_id'] == emp_id]
    top5 = emp_strengths[emp_strengths['rank'] <= 5]
    
    if top5.empty:
        return {
            'behavioral_score': 0,
            'tv_details': []
        }
    
    themes = set(top5['theme'].dropna())
    
    thinker_themes = {'Intellection', 'Analytical', 'Strategic', 'Futuristic'}
    doer_themes = {'Activator', 'Responsibility', 'Self-Assurance', 'Belief'}
    
    has_thinker = len(themes & thinker_themes) > 0
    has_doer = len(themes & doer_themes) > 0
    
    tv_details = []
    match_rates = []
    
    # Thinker Cluster
    thinker_match = 100 if has_thinker else 0
    match_rates.append(thinker_match)
    tv_details.append({
        'tv_code': 'Thinker_Cluster',
        'tv_name': 'Thinker Cluster (Intellection, Analytical, Strategic, Futuristic)',
        'user_score': 1 if has_thinker else 0,
        'baseline_score': 1,
        'tv_match_rate': thinker_match
    })
    
    # Doer Cluster
    doer_match = 100 if has_doer else 0
    match_rates.append(doer_match)
    tv_details.append({
        'tv_code': 'Doer_Cluster',
        'tv_name': 'Doer Cluster (Activator, Responsibility, Self-Assurance, Belief)',
        'user_score': 1 if has_doer else 0,
        'baseline_score': 1,
        'tv_match_rate': doer_match
    })
    
    behav_score = round(np.mean(match_rates), 2) if match_rates else 0
    
    return {
        'behavioral_score': behav_score,
        'tv_details': tv_details
    }


def calculate_contextual_match(emp_id):
    """
    Calculate TGV match for Contextual Fit (5% weight).
    
    - 2 TVs: Grade_Fit, Tenure_Fit
    - Grade: III-IV optimal (100%), V acceptable (75%), others (50%)
    - Tenure: 2-5 years optimal (100%), 1-6 years acceptable (75%), others (50%)
    - Average of TV match rates = TGV match rate
    """
    emp_data = employees[employees['employee_id'] == emp_id]
    
    if emp_data.empty:
        return {
            'contextual_score': 50,
            'tv_details': []
        }
    
    grade = emp_data['grade_id'].iloc[0]
    tenure_months = emp_data['years_of_service_months'].iloc[0]
    
    tv_details = []
    match_rates = []
    
    # Grade Fit
    if pd.notna(grade):
        grade_fit = 100 if grade in [1, 2] else (75 if grade == 3 else 50)
        match_rates.append(grade_fit)
        tv_details.append({
            'tv_code': 'Grade_Fit',
            'tv_name': 'Grade Level',
            'user_score': grade,
            'baseline_score': 3,  # Grade III baseline
            'tv_match_rate': grade_fit
        })
    
    # Tenure Fit
    if pd.notna(tenure_months):
        if 24 <= tenure_months <= 60:
            tenure_fit = 100
        elif 12 <= tenure_months <= 72:
            tenure_fit = 75
        else:
            tenure_fit = 50
        match_rates.append(tenure_fit)
        tv_details.append({
            'tv_code': 'Tenure_Fit',
            'tv_name': 'Tenure (Years)',
            'user_score': tenure_months,
            'baseline_score': 36,  # 3 years baseline
            'tv_match_rate': tenure_fit
        })
    
    context_score = round(np.mean(match_rates), 2) if match_rates else 50
    
    return {
        'contextual_score': context_score,
        'tv_details': tv_details
    }


print("  ✓ Match calculation functions defined")
print("    - Competency Match (50%): 10 TV pillars")
print("    - Psychometric Match (25%): PAULI 60%, GTQ 28%, IQ 12%")
print("    - Behavioral Match (20%): Thinker/Doer clusters")
print("    - Contextual Match (5%): Grade/Tenure fit")

  ✓ Match calculation functions defined
    - Competency Match (50%): 10 TV pillars
    - Psychometric Match (25%): PAULI 60%, GTQ 28%, IQ 12%
    - Behavioral Match (20%): Thinker/Doer clusters
    - Contextual Match (5%): Grade/Tenure fit


## STEP 3: Calculate Match Scores for All Employees

In [7]:
all_employees = perf_2025['employee_id'].unique()
results = []
tv_breakdown = []

for i, emp_id in enumerate(all_employees):
    if (i + 1) % 500 == 0:
        print(f"    Progress: {i + 1}/{len(all_employees)}")
    
    # Calculate TGV scores
    comp_result = calculate_competency_match(emp_id)
    psych_result = calculate_psychometric_match(emp_id)
    behav_result = calculate_behavioral_match(emp_id)
    context_result = calculate_contextual_match(emp_id)
    
    comp_score = comp_result['competency_score']
    psych_score = psych_result['psychometric_score']
    behav_score = behav_result['behavioral_score']
    context_score = context_result['contextual_score']
    
    # Calculate Final Match Score (Weighted Average)
    # Weights: Competency 50%, Psychometric 25%, Behavioral 20%, Contextual 5%
    final_score = (
        comp_score * 0.50 +
        psych_score * 0.25 +
        behav_score * 0.20 +
        context_score * 0.05
    )
    
    results.append({
        'employee_id': emp_id,
        'competency_score': comp_score,
        'psychometric_score': psych_score,
        'behavioral_score': behav_score,
        'contextual_score': context_score,
        'final_match_score': round(final_score, 2)
    })
    
    # Store TV breakdown for detailed analysis
    for tv_detail in comp_result['tv_details']:
        tv_breakdown.append({
            'employee_id': emp_id,
            'tgv': 'Competency_Excellence',
            'tv_code': tv_detail['tv_code'],
            'tv_name': tv_detail['tv_name'],
            'user_score': tv_detail['user_score'],
            'baseline_score': tv_detail['baseline_score'],
            'tv_match_rate': tv_detail['tv_match_rate']
        })
    
    for tv_detail in psych_result['tv_details']:
        tv_breakdown.append({
            'employee_id': emp_id,
            'tgv': 'Cognitive_Ability',
            'tv_code': tv_detail['tv_code'],
            'tv_name': tv_detail['tv_name'],
            'user_score': tv_detail['user_score'],
            'baseline_score': tv_detail['baseline_score'],
            'tv_match_rate': tv_detail['tv_match_rate']
        })
    
    for tv_detail in behav_result['tv_details']:
        tv_breakdown.append({
            'employee_id': emp_id,
            'tgv': 'Behavioral_Strengths',
            'tv_code': tv_detail['tv_code'],
            'tv_name': tv_detail['tv_name'],
            'user_score': tv_detail['user_score'],
            'baseline_score': tv_detail['baseline_score'],
            'tv_match_rate': tv_detail['tv_match_rate']
        })
    
    for tv_detail in context_result['tv_details']:
        tv_breakdown.append({
            'employee_id': emp_id,
            'tgv': 'Contextual_Fit',
            'tv_code': tv_detail['tv_code'],
            'tv_name': tv_detail['tv_name'],
            'user_score': tv_detail['user_score'],
            'baseline_score': tv_detail['baseline_score'],
            'tv_match_rate': tv_detail['tv_match_rate']
        })

match_df = pd.DataFrame(results).sort_values('final_match_score', ascending=False)
tv_breakdown_df = pd.DataFrame(tv_breakdown)

print(f"  Match scores calculated for {len(match_df)} employees")
print(f"  TV details captured: {len(tv_breakdown_df)} records")

    Progress: 500/2010
    Progress: 1000/2010
    Progress: 1500/2010
    Progress: 2000/2010
  Match scores calculated for 2010 employees
  TV details captured: 33382 records


## STEP 4: Add Employee Context & Categories

In [8]:
# Merge with employee details
final_ranking = match_df.merge(
    employees[['employee_id', 'fullname', 'grade_id']],
    on='employee_id',
    how='left'
)
final_ranking = final_ranking.merge(
    dim_grades,
    on='grade_id',
    how='left',
    suffixes=('', '_grade')
)
final_ranking = final_ranking.merge(
    perf_2025[['employee_id', 'rating']],
    on='employee_id',
    how='left'
)

# Add match category
def get_category(score):
    """Classify match score into category."""
    if score >= 80:
        return 'Excellent (80-100%)'
    elif score >= 60:
        return 'Good (60-79%)'
    elif score >= 40:
        return 'Moderate (40-59%)'
    else:
        return 'Low (0-39%)'

final_ranking['match_category'] = final_ranking['final_match_score'].apply(get_category)

# Add recommendation
def get_recommendation(score):
    """Generate recommendation based on match score."""
    if score >= 80:
        return 'Ready for succession/promotion'
    elif score >= 60:
        return 'High-potential development'
    elif score >= 40:
        return 'Targeted development needed'
    else:
        return 'Role reassessment'

final_ranking['recommendation'] = final_ranking['final_match_score'].apply(get_recommendation)

print("  Employee context added")
print("  Match categories assigned")
print("  Recommendations generated")

  Employee context added
  Match categories assigned
  Recommendations generated


## STEP 5: Display Top 20 Talent Matches

In [10]:
display_df = final_ranking[[
    'employee_id', 'fullname', 'final_match_score', 'match_category',
    'competency_score', 'psychometric_score', 'behavioral_score', 'contextual_score',
    'name', 'rating', 'recommendation'
]].copy()

display_df.columns = [
    'Employee ID', 'Name', 'Match Score', 'Category',
    'Competency', 'Psychometric', 'Behavioral', 'Contextual',
    'Grade', 'Current Rating', 'Recommendation'
]

print(display_df.head(20).to_string(index=False))

Employee ID                      Name  Match Score            Category  Competency  Psychometric  Behavioral  Contextual Grade  Current Rating                 Recommendation
  EMP100761           Maya Situmorang        97.77 Excellent (80-100%)       96.34         98.40       100.0       100.0    IV             5.0 Ready for succession/promotion
  EMP101396              Gita Ginting        96.76 Excellent (80-100%)       96.24         94.57       100.0       100.0   III             5.0 Ready for succession/promotion
  EMP101226    Yoga Erlangga Mahendra        96.67 Excellent (80-100%)       96.34         93.99       100.0       100.0   III             5.0 Ready for succession/promotion
  EMP100838           Nurlita Permadi        96.40 Excellent (80-100%)       96.35         92.91       100.0       100.0    IV             NaN Ready for succession/promotion
  EMP100924   Julia Yudhistira Lukman        96.32 Excellent (80-100%)       96.15         95.47       100.0        87.5     V    

## STEP 6: Detailed TV Breakdown For TOP 10

In [12]:
for rank, (idx, row) in enumerate(final_ranking.head(10).iterrows(), 1):
    emp_id = row['employee_id']
    emp_name = row['fullname']
    emp_score = row['final_match_score']
    
    print(f"\n{rank}. {emp_name} (ID: {emp_id}) - Final Score: {emp_score}%")
    print("-" * 80)
    
    emp_tv = tv_breakdown_df[tv_breakdown_df['employee_id'] == emp_id]
    
    # Group by TGV
    for tgv in ['Competency_Excellence', 'Cognitive_Ability', 'Behavioral_Strengths', 'Contextual_Fit']:
        tgv_data = emp_tv[emp_tv['tgv'] == tgv]
        if not tgv_data.empty:
            tgv_avg = tgv_data['tv_match_rate'].mean()
            print(f"\n  {tgv} (Avg: {round(tgv_avg, 2)}%)")
            for _, tv in tgv_data.iterrows():
                print(f"    • {tv['tv_name']:50} {tv['tv_match_rate']:6.2f}%")


1. Maya Situmorang (ID: EMP100761) - Final Score: 97.77%
--------------------------------------------------------------------------------

  Competency_Excellence (Avg: 96.34%)
    • GDR                                                 92.47%
    • CEX                                                 94.52%
    • IDS                                                 94.46%
    • QDD                                                 95.08%
    • STO                                                 99.85%
    • SEA                                                 99.69%
    • VCU                                                 99.74%
    • LIE                                                 93.82%
    • FTC                                                 94.08%
    • CSI                                                 99.69%

  Cognitive_Ability (Avg: 96.6%)
    • PAULI - Mental Speed & Accuracy                     99.84%
    • GTQ - General Aptitude Test                         98.14%
    • IQ

## STEP 6: Match Score Distribution

In [13]:
category_dist = final_ranking['match_category'].value_counts().sort_index()
print("Category Distribution:")
print(category_dist)

print("\nPercentage Breakdown:")
for cat in ['Excellent (80-100%)', 'Good (60-79%)', 'Moderate (40-59%)', 'Low (0-39%)']:
    count = len(final_ranking[final_ranking['match_category'] == cat])
    pct = (count / len(final_ranking)) * 100
    print(f"  {cat:20} {count:5} employees ({pct:5.1f}%)")

Category Distribution:
match_category
Excellent (80-100%)     662
Good (60-79%)          1286
Moderate (40-59%)        62
Name: count, dtype: int64

Percentage Breakdown:
  Excellent (80-100%)    662 employees ( 32.9%)
  Good (60-79%)         1286 employees ( 64.0%)
  Moderate (40-59%)       62 employees (  3.1%)
  Low (0-39%)              0 employees (  0.0%)


## STEP 7: Summary Statistics

In [14]:
print("\n Summary Statistics\n")

print(f"Total Employees Analyzed: {len(match_df)}")
print(f"\nFinal Match Score Distribution:")
print(f"  Average: {match_df['final_match_score'].mean():.2f}%")
print(f"  Median:  {match_df['final_match_score'].median():.2f}%")
print(f"  Min:     {match_df['final_match_score'].min():.2f}%")
print(f"  Max:     {match_df['final_match_score'].max():.2f}%")
print(f"  Std Dev: {match_df['final_match_score'].std():.2f}%")

print(f"\nTalent Pool Breakdown:")
excellent = len(final_ranking[final_ranking['final_match_score'] >= 80])
good = len(final_ranking[(final_ranking['final_match_score'] >= 60) & (final_ranking['final_match_score'] < 80)])
moderate = len(final_ranking[(final_ranking['final_match_score'] >= 40) & (final_ranking['final_match_score'] < 60)])
low = len(final_ranking[final_ranking['final_match_score'] < 40])

print(f"  Excellent (80-100%): {excellent} employees")
print(f"    → Ready for succession/promotion")
print(f"  Good (60-79%):       {good} employees")
print(f"    → High-potential development")
print(f"  Moderate (40-59%):   {moderate} employees")
print(f"    → Requires targeted development")
print(f"  Low (0-39%):         {low} employees")
print(f"    → Consider role reassessment")


 Summary Statistics

Total Employees Analyzed: 2010

Final Match Score Distribution:
  Average: 75.97%
  Median:  75.82%
  Min:     48.92%
  Max:     97.77%
  Std Dev: 8.86%

Talent Pool Breakdown:
  Excellent (80-100%): 662 employees
    → Ready for succession/promotion
  Good (60-79%):       1286 employees
    → High-potential development
  Moderate (40-59%):   62 employees
    → Requires targeted development
  Low (0-39%):         0 employees
    → Consider role reassessment
